In [1]:
!pip install transformers sentence-transformers datasets cohere pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
!pip install pinecone-client

In [3]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from pinecone import Pinecone, ServerlessSpec
import os
from tqdm import tqdm
import cohere
import numpy as np
import warnings
from IPython.display import display
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
with open("/content/chohere_api_keys.txt") as f:
    COHERE_API_KEY = f.read().strip()
with open("/content/pinecone_api_key.txt") as f:
    PINECONE_API_KEY = f.read().strip()

In [8]:
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL = 'all-MiniLM-L6-v2'
model = SentenceTransformer(EMBEDDING_MODEL)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
def load_and_embedd_dataset(
        dataset_name: str = 'RealTimeData/News_August_2023',
        split: str = 'train',
        model: SentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2'),
        text_field: str = 'description',
        rec_num: int = 400
) -> tuple:
    """
    Load a dataset and embedd the text field using a sentence-transformer model
    Args:
        dataset_name: The name of the dataset to load
        split: The split of the dataset to load
        model: The model to use for embedding
        text_field: The field in the dataset that contains the text
        rec_num: The number of records to load and embedd
    Returns:
        tuple: A tuple containing the dataset and the embeddings
    """
    from datasets import load_dataset

    print("Loading and embedding the dataset")

    # Load the dataset
    dataset = load_dataset(dataset_name, split=split)

    # Embed the first `rec_num` rows of the dataset
    embeddings = model.encode(dataset[text_field][:rec_num])

    print("Done!")
    return dataset, embeddings

In [10]:
DATASET_NAME = 'RealTimeData/News_August_2023'

dataset, embeddings = load_and_embedd_dataset(
    dataset_name=DATASET_NAME,
    rec_num=1000,
    model=model,
)
shape = embeddings.shape

Loading and embedding the dataset


Generating train split:   0%|          | 0/5059 [00:00<?, ? examples/s]

Done!


In [11]:
pd_dataset = dataset.to_pandas()
pd_dataset.head(5)

,authors,date_download,date_modify,date_publish,description,filename,image_url,language,localpath,maintext,source_domain,title,title_page,title_rss,url
0,[],2023-08-01 01:20:55+00:00,_,2023-08-01 01:10:02,A consultant cardiologist at the Federal Medic...,https%3A%2F%2Fdailytrust.com%2Ftherapeutic-lif...,https://dailytrust.com/wp-content/uploads/2018...,en,_,A consultant cardiologist at the Federal Medic...,dailytrust.com,‘Therapeutic lifestyle modification’ lowers ri...,_,_,https://dailytrust.com/therapeutic-lifestyle-m...
1,[],2023-08-01 01:20:06+00:00,_,2023-08-01 01:13:54,The Nasarawa State government is taking measur...,https%3A%2F%2Fdailytrust.com%2Fhow-nasarawa-go...,https://dailytrust.com/wp-content/uploads/2022...,en,_,The Nasarawa State government is taking measur...,dailytrust.com,How Nasarawa govt is responding to diphtheria ...,_,_,https://dailytrust.com/how-nasarawa-govt-is-re...
2,[],2023-08-01 01:20:19+00:00,_,2023-08-01 01:07:57,Lawyers are divided over the renewed moves to ...,https%3A%2F%2Fdailytrust.com%2Fnba-conference-...,https://dailytrust.com/wp-content/uploads/2022...,en,_,Lawyers are divided over the renewed moves to ...,dailytrust.com,NBA Conference: Lawyers divided over parallel ...,_,_,https://dailytrust.com/nba-conference-lawyers-...
3,[],2023-08-01 01:20:00+00:00,_,2023-08-01 00:37:29,D’Tigress will face the winners between Mozamb...,https%3A%2F%2Fdailytrust.com%2Fdtigress-to-fac...,https://dailytrust.com/wp-content/uploads/2022...,en,_,D’Tigress will face the winners between Mozamb...,dailytrust.com,D’Tigress to face Mozambique or Cote d’Ivoire ...,_,_,https://dailytrust.com/dtigress-to-face-mozamb...
4,[],2023-08-01 01:20:37+00:00,_,2023-08-01 01:11:50,Liver cancer patients are being spared overnig...,https%3A%2F%2Fdailytrust.com%2Fradioactive-bea...,https://dailytrust.com/wp-content/uploads/2022...,en,_,Liver cancer patients are being spared overnig...,dailytrust.com,Radioactive beads in the wrist that can fight ...,_,_,https://dailytrust.com/radioactive-beads-in-th...


In [12]:
print(f"The embeddings shape: {embeddings.shape}")

The embeddings shape: (1000, 384)


In [13]:
def create_pinecone_index(
        index_name: str,
        dimension: int,
        metric: str = 'cosine',
):
    """
    Create a pinecone index if it does not exist
    Args:
        index_name: The name of the index
        dimension: The dimension of the index
        metric: The metric to use for the index
    Returns:
        Pinecone: A pinecone object which can later be used for upserting vectors and connecting to VectorDBs
    """
    from pinecone import Pinecone, ServerlessSpec
    print("Creating a Pinecone index...")
    pc = Pinecone(api_key=PINECONE_API_KEY)
    existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
    if index_name not in existing_indexes:
        pc.create_index(
            name=index_name,
            dimension=dimension,
            # Remember! It is crucial that the metric you will use in your VectorDB will also be a metric your embedding
            # model works well with!
            metric=metric,
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
    print("Done!")
    return pc

In [14]:
INDEX_NAME = 'news-august-2023'

# Create the vector database
# We are passing the index_name and the size of our embeddings
pc = create_pinecone_index(INDEX_NAME, shape[1])

Creating a Pinecone index...
Done!


In [15]:
def upsert_vectors(
        index: Pinecone,
        embeddings: np.ndarray,
        dataset: dict,
        text_field: str = 'description',
        batch_size: int = 128
):
    """
    Upsert vectors to a pinecone index
    Args:
        index: The pinecone index object
        embeddings: The embeddings to upsert
        dataset: The dataset containing the metadata
        batch_size: The batch size to use for upserting
    Returns:
        An updated pinecone index
    """
    print("Upserting the embeddings to the Pinecone index...")
    shape = embeddings.shape

    ids = [str(i) for i in range(shape[0])]
    meta = [{text_field: text} for text in dataset[text_field]]

    # create list of (id, vector, metadata) tuples to be upserted
    to_upsert = list(zip(ids, embeddings, meta))

    for i in tqdm(range(0, shape[0], batch_size)):
        i_end = min(i + batch_size, shape[0])
        index.upsert(vectors=to_upsert[i:i_end])
    return index


In [16]:
# Upsert the embeddings to the Pinecone index
index = pc.Index(INDEX_NAME)
index_upserted = upsert_vectors(index, embeddings, dataset)

Upserting the embeddings to the Pinecone index...


100%|██████████| 8/8 [00:07<00:00,  1.04it/s]


In [17]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}

In [18]:
import cohere

#First lets write a query for the LLM
query = "England beat Australia by how many runs in the fifth and final Ashes Test at The Oval in 2023?"

co = cohere.Client(api_key=COHERE_API_KEY)
response = co.chat(
        model='command-r-plus',
        message=query,
    )
response.text

'England beat Australia by 192 runs in the fifth and final Ashes Test at The Oval in September 2023. This victory ensured that the series ended in a 3-3 draw.'

In [19]:
def augment_prompt(
        query: str,
        model: SentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2'),
        index=None,
) -> str:
    """
    Augment the prompt with the top 3 results from the knowledge base
    Args:
        query: The query to augment
        index: The vectorstore object
    Returns:
        str: The augmented prompt
    """
    results = [float(val) for val in list(model.encode(query))]

    # get top 3 results from knowledge base
    query_results = index.query(
        vector=results,
        top_k=3,
        include_values=True,
        include_metadata=True
    )['matches']
    text_matches = [match['metadata']['description'] for match in query_results]

    # get the text from the results
    source_knowledge = "\n\n".join(text_matches)

    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.
    Contexts:
    {source_knowledge}
    If the answer is not included in the source knowledge - say that you don't know.
    Query: {query}"""
    return augmented_prompt, source_knowledge

In [20]:
# Let us remember our query
query = "England beat Australia by how many runs in the fifth and final Ashes Test at The Oval in 2023?"
augmented_prompt, source_knowledge = augment_prompt(query, model=model, index=index)
response = co.chat(
        model='command-r-plus',
        message=augmented_prompt,
    )
response.text

'England beat Australia by 49 runs in the fifth and final Ashes Test at The Oval in 2023.'

In [21]:
print(source_knowledge)

Home SportsAshes 2023: Stuart Broad Gets Winning Send-Off As England Beat Australia By 49 Runs; Draw Series 2-2 Veteran fast-bowler Stuart Broad got his perfect fairytale ending to a glorious cricketing career by picking the last two wickets as England beat Australia by 49 runs in the fifth and final Ashes Test at The Oval…

You get three chances to correctly rank the answers in today's question for August 1, 2023.

Three New Zealanders have grabbed back some Kiwi pride with wins at the glamour event.


In [24]:

# another example :
# cohere
query_1 = "What specific issue puts Wairarapa Hospital at risk during a major earthquake?"
co = cohere.Client(api_key=COHERE_API_KEY)
response = co.chat(
        model='command-r-plus',
        message=query_1,
    )
response.text

"The specific issue that puts Wairarapa Hospital at risk during a major earthquake is its proximity to the Wellington Fault Line, which is one of the most active and dangerous fault lines in New Zealand. The hospital is located in the town of Masterton, which is in the Wairarapa region, just north of Wellington. This region is known for its high seismic activity and has a high risk of experiencing strong earthquakes.\n\nThe Wellington Fault Line poses a significant threat to the hospital's infrastructure and the safety of its patients and staff. During a major earthquake, the fault line could generate strong ground shaking, soil liquefaction, and surface rupturing, which could severely damage the hospital's buildings and disrupt its critical services.\n\nThe age and structural design of the hospital buildings are also factors that contribute to the risk. Some of the hospital's buildings are older and may not meet the latest seismic standards, making them more vulnerable to damage or co

In [23]:
augmented_prompt, source_knowledge = augment_prompt(query_1, model=model, index=index)
response = co.chat(
        model='command-r-plus',
        message=augmented_prompt,
    )
response.text

'Wairarapa Hospital is at risk during a major earthquake due to the almost 3000 structural defects in the building, which means it does not comply with the building code.'

In [25]:
print(source_knowledge)

Wairarapa Hospital could be at risk of failing in a major earthquake, with almost 3000 structural defects found in the building, which does not comply with the building code.

A reported shooting outside Baptist Memorial Hospital-Golden Triangle has the hospital on lockdown.

Experts say a combination of mitigatable risks and loose safety regulations are contributing to deaths.
